## HTML and the DOM
The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML
underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

In [ ]:
from bs4 import BeautifulSoup
import requests
response = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')
soup = BeautifulSoup(response.text, "lxml")
print(type(soup))

The soup object contains all of the HTML in the original document.

In [ ]:
print(soup.prettify()[0:2000])

The HTML tags contained in the angled brackets provide structural information (and sometimes formatting), which we probably don't care about in and of itself but is useful for selecting only the content relevant to our needs.

Beautiful Soup is essentially a set of wrapper functions that make it simple to select common HTML elements.

In [ ]:
from IPython.display import Image
Image('http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png')

Most modern browsers have a parser that reads in the HTML document, parses it into a DOM (Document Object Model) structure, and then renders the DOM structure.

Much like HTTP, the DOM is an agreed-upon standard.

The DOM is much more than what I've described, but for our purposes, what is most important to understand is that the text is only one part of an HTML element, and we need to select it explicitly.

In [ ]:
Image('http://www.cs.toronto.edu/~shiva/cscb07/img/dom/treeStructure.png')

## Parsing HTML
I study interest group political activity, and I'd like to know what issues organized labor was active on this year. Fortunately, the largest American labor federation has a website where they document the letters* that the national organization has sent to federal legislators. This is the page we loaded in above.*

Some things to always think about, regardless of your source: Who created this website? Do we think this is likely to be a complete list? Might this organization have some reason to selectively post information?

Look at the following link :
[http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts](http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts
)

Our goal here is to get the name of each item in this list, the URL that it links to, and the associated date. This sounds easy, but there is alot of junk to wade through to find what we want. By looking at the source code, I found that all of the items I am interested in are in a single div element of class "legisalerts_listing", and that each individual item is a member of class "ec_statements". Here is what the first few look like in HTML.

In [ ]:
print(soup.prettify()[28700:30500])

I can use the heirarchical nature of HTML structure to grab precisely the content that I am interested in.  
I will grab all of the elements that are within div tags and are also members of class "ec_statements."

In [ ]:
letters = soup.find_all("div", class_="content-details")

This returns a collection of tag objects. This is not one of the normal Python collections covered in the other tutorials, it is an object specific to the Beautiful Soup library. It can be iterated over, but most other standard methods won't work on it. We'll have to do some preprocessing to get out the content that we want.

In [ ]:
print(type(letters))

By examining one of the elements in this collection, we can see that the information we want inside these objects, but we'll need to use more of Beautiful Soup's functionality and know something about HTML strucure to access them.

In [ ]:
letters[0]

Recall that we want 3 things: the text of the item as it appears on the website, the URL that is linked to (so we can scrape and analyze it later), and the date the letter was sent.

First we should consider how we are going to store this data. Since we want to maintain the association between the 3 things from each observation. A natural way to store this is as a nested dict. Dict keys must be unique, and some of our items have the same associated date, so we'll have to use one of the other items. We'll use the name.

Visible text is always placed between tags. On the rendered page, the name we want is an active link, but we can get just the associated text by using the get_text method on the a tags. You can also use contents, which returns a list instead of a string.

We'll go through all of the items in our letters collection, and for each one, pull out the name and make it a key in our dict. The value will be another dict, but we haven't yet found the contents for the other items yet so we'll just create assign an empty dict object.

In [ ]:
print(letters[0].a.span)
print(letters[0].a.span.get_text())

In [ ]:
lobbying = {}
for letter in letters:
    key = letter.a.span.get_text()
    lobbying[key] = {}

Now we want to add to our inner dictionary by the linked URL.

In HTML, links are always enclosed in a tags as the href attribute. We can use this fact to easily pull out the links. But if we look at the URLs, we'll find that they are incomplete. Like many websites, this site uses relative paths.

In [ ]:
letters[0].a["href"]

If we want to visit these links later we'll need the complete path, so we will add in a prefix to complete these paths.

In [ ]:
prefix = "www.aflcio.org"

Now we'll go through our letters collection again, and add a link key to our lobbying dictionaries.

In [ ]:
for letter in letters:
    key = letter.a.span.get_text()
    link = prefix + letter.a["href"]
    lobbying[key]["link"] = link

In [ ]:
for item in lobbying.items():
    print(item)

Now we'll do the same thing with the date. Note that all of the dates are found within time tags.  
We can use this structure to grab the date information just as we did the link text.  
We know there is only one of these per item so we can use find instead of find_all.

In [ ]:
print(letters[0].find('time'))
print(letters[0].find('time').get_text())

Now we'll add it into our lobbying dictionary. We can see above that we only want the text between the tags, so we'll pull that out before adding it into the dictionary.

In [ ]:
for letter in letters:
    key = letter.a.span.get_text()
    date = letter.time.get_text()
    lobbying[key]["date"] = date

Let's look at what we've made.

In [ ]:
for item in lobbying.keys():
    print()
    print(item + ": ")
    print("  link: " + lobbying[item]["link"])
    print("  date: " + lobbying[item]["date"]) 

## Writing to File

Now we can save it for later use. Typically you will use the DictWriter class of the csv module to write nested dictionaries, however if you want to write the keys as well as the values, you can't use that package. Because we want our keys in this case, we'll use the more general version

In [ ]:
import os, csv

with open("lobbying.csv", "w") as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow(["name", "link", "date"])
    for a in lobbying.keys():
        writer.writerow([a.encode("utf-8"), lobbying[a]["link"], lobbying[a]["date"]])

A better format for recording nested dictionaries is JSON (Java Script Object Notation).  
It is trivially easy to write this type of structure with nested dictionaries in Python.  
This is a format you should get comfortable with, because this is how most data from APIs is provided.  
But we'll save that for another tutorial!

In [ ]:
import json

with open("lobbying.json", "w") as writeJSON:
    json.dump(lobbying, writeJSON)